In [ ]:
import sys
import os
import numpy as np
import openmm as mm
from openmm import unit
from openmm.app import PDBFile
from openmm.app.dcdfile import DCDFile
from openmmtools.multistate import MultiStateReporter, ReplicaExchangeAnalyzer
import pdb_numpy
from REST2_initial import run_rest2_hrex
import mdtraj as md

sys.path.append(os.path.abspath('..'))
from src.useful_functions import *


In [ ]:
# Read directory paths
read_dirs_paths('dir_paths.txt', globals())
check_directories(out_dir ,out_trajectories2 ,out_trajectories3 ,out_trajectories4)

In [ ]:
from openmm.app import PDBFile, ForceField
from openmm import unit

# Load your reference system
pdb = PDBFile(inp_dir + "pdbfile_water_1.pdb")
ff  = ForceField("amber14-all.xml","amber14/tip3pfb.xml")
system_ref = ff.createSystem(pdb.topology, nonbondedMethod=mm.app.PME,
                             nonbondedCutoff=1.0*unit.nanometer, constraints=mm.app.HBonds)

# Define solute atom indices (e.g., all protein atoms)
prot_pep_coor = pdb_numpy.Coor(inp_dir + "pdbfile_water_1.pdb")
solute_indices = prot_pep_coor.get_index_select("chain A")


# Ladder (all replicas at 300 K; s from 1.0 down to, say, 0.6)
nrep = 12
s_list = np.linspace(1.0, 0.95, nrep).tolist()
temperatures = [300.0*unit.kelvin]*nrep

run_rest2_hrex(inp_dir + "pdbfile_water_1.pdb", system_ref, solute_indices, temperatures, s_list,
               nsteps=25000, swap_interval=250, timestep=2.0*unit.femtoseconds)
